In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('C:/Users/bhatt/OneDrive/Desktop/whisper-main/ANN_Data/data1.csv', header=None)
print(data)
labels=pd.read_csv('C:/Users/bhatt/OneDrive/Desktop/whisper-main/ANN_Data/label1.csv',header=None)
print(labels)

             0         1         2         3         4         5         6   \
0     -0.446891 -0.013397  0.232645  2.156649  1.652923 -0.210531 -0.662227   
1      0.243747 -1.144175 -0.622214 -0.661979 -0.373315  0.520313 -1.307954   
2     -1.417799 -0.088833 -0.647181 -1.141497 -1.321100  0.686798 -1.045569   
3      0.656784  0.381201 -1.039011  1.285315  0.808324  1.428519 -1.144367   
4      1.098520  1.706769 -1.030370  2.001009  2.751551  1.202229 -0.941066   
...         ...       ...       ...       ...       ...       ...       ...   
19995  0.146324 -0.618555  0.547413  0.851301  0.838880 -1.248402  0.698199   
19996 -2.227923 -0.327196  1.258849 -0.945254  3.925262  0.316686 -1.061159   
19997 -0.800405  1.477997  0.090057 -0.973997 -0.204970  0.224539  0.302265   
19998 -1.217344 -0.785401 -0.413781 -0.335324  0.181671 -1.685733 -0.437981   
19999  1.601005 -0.890398 -1.425049  0.887501  0.167205 -0.395042 -0.512554   

             7         8         9         10      

In [3]:
data=data.values
npLabels=labels.values
print(data.shape)
print(npLabels.shape)

(20000, 20)
(20000, 1)


In [4]:
print(np.max(npLabels))
print(np.min(npLabels))

4
0


In [5]:
np.unique(npLabels)

array([0, 1, 2, 3, 4], dtype=int64)

In [6]:
num=np.max(npLabels)+1
oneHot=np.eye(num)[npLabels]
print(oneHot.ndim)
print(oneHot.shape)

3
(20000, 1, 5)


In [7]:
oneHotRe=oneHot.reshape(20000, 5)
print(oneHotRe)

[[0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 ...
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]]


In [8]:
trainData,testDataDash=train_test_split(data,train_size=0.8,test_size=0.2,shuffle=False)
trainLabel,testLabelDash=train_test_split(oneHotRe,train_size=0.8,test_size=0.2,shuffle=False)

In [9]:
validData,testData=train_test_split(testDataDash,train_size=0.5,test_size=0.5,shuffle=False)
validLabel,testLabel=train_test_split(testLabelDash,train_size=0.5,test_size=0.5,shuffle=False)

In [10]:
h = 15

In [11]:
b1=np.zeros(h)
b2=np.zeros(5)
print(b1)
print(b2)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]


In [12]:
w1=np.random.normal(0, 1, (20,h))
w2=np.random.normal(0, 1, (h,5))

In [13]:
def actFunc(data,choice):

    if(choice==1):
        return np.tanh(data)
    elif(choice==2):
        numerator=np.exp(data)
        return numerator/np.sum(numerator,axis=1,keepdims=True)

In [14]:
alpha = 0.003

In [15]:
batch=1000

In [16]:
def costFunc(y,t):
    return -(t*(np.log(y))+(1-t)*np.log(1-y))

In [17]:
def forwFunc(data):
    a0=data
    z1=np.dot(a0,w1)+b1
    a1=actFunc(z1,1)
    z2=np.dot(a1,w2)+b2
    a2=actFunc(z2,2)
    return a0,z1,a1,z2,a2

In [18]:
def Acc(y,t,size):
    maxData=np.argmax(y,axis=1)
    maxLabel=np.argmax(t,axis=1)
    compare=np.equal(maxData,maxLabel)
    count=np.sum(compare)
    return (count/size)*100

In [19]:
epochs=500

In [20]:
for epoch in range(epochs):
    for iteration in range(int(len(trainData)/batch)):
        a0,z1,a1,z2,a2=forwFunc(trainData[iteration*batch:(iteration+1)*batch,:])
        y=a2
        labelBatch=trainLabel[iteration*batch:(iteration+1)*batch,:]
        del2=(y-labelBatch)
        del1=np.dot(del2,w2.T)*(1 - a1**2)
        dcdw2=np.dot(a1.T,del2)
        dcdw1=np.dot(a0.T,del1)
        dcdb1=np.sum(del1,axis=0)
        dcdb2=np.sum(del2,axis=0)
        w1=w1-alpha*dcdw1
        w2=w2-alpha*dcdw2
        b2=b2-alpha*dcdb2
        b1=b1-alpha*dcdb1
    a0,z1,a1,z2,a2=forwFunc(trainData)
    print("Training Cost",(np.sum(costFunc(a2,trainLabel)))/16000.0)
    print("Training accuracy",Acc(a2,trainLabel,len(trainLabel)))
    
va0,vz1,va1,vz2,va2=forwFunc(validData)
vOutput=va2
accuracy = Acc(vOutput,validLabel,2000.0)
print("validation accuracy")
print(accuracy)

Training Cost 1.3360926734520475
Training accuracy 73.1375
Training Cost 1.0685016728695331
Training accuracy 78.875
Training Cost 0.9401474834529392
Training accuracy 81.975
Training Cost 0.8870572230564632
Training accuracy 83.00625000000001
Training Cost 0.843930172184744
Training accuracy 83.6625
Training Cost 0.8106265917266584
Training accuracy 84.28750000000001
Training Cost 0.7888949165870727
Training accuracy 84.8375
Training Cost 0.7651297108538834
Training accuracy 85.375
Training Cost 0.7527190762207108
Training accuracy 85.46875
Training Cost 0.7303304502333449
Training accuracy 85.98125
Training Cost 0.7279308265371006
Training accuracy 86.0
Training Cost 0.688887781073648
Training accuracy 87.0625
Training Cost 0.6953550204322305
Training accuracy 86.8625
Training Cost 0.6936150350970636
Training accuracy 86.93125
Training Cost 0.681301059052426
Training accuracy 87.10625
Training Cost 0.6739675408230308
Training accuracy 87.25625000000001
Training Cost 0.667629615066533

Training Cost 0.4917445357276797
Training accuracy 91.20625
Training Cost 0.49153309280531476
Training accuracy 91.20625
Training Cost 0.49132313175604897
Training accuracy 91.21249999999999
Training Cost 0.49111519066491127
Training accuracy 91.19375
Training Cost 0.4909100465685939
Training accuracy 91.19375
Training Cost 0.4907085723557612
Training accuracy 91.19375
Training Cost 0.4905115911230975
Training accuracy 91.21249999999999
Training Cost 0.49031974377566395
Training accuracy 91.21875
Training Cost 0.4901334086110909
Training accuracy 91.21875
Training Cost 0.48995269498699584
Training accuracy 91.2
Training Cost 0.48977749170873824
Training accuracy 91.21249999999999
Training Cost 0.48960753121438594
Training accuracy 91.20625
Training Cost 0.4894424439524365
Training accuracy 91.21875
Training Cost 0.4892817978808576
Training accuracy 91.23125
Training Cost 0.4891251276404165
Training accuracy 91.23750000000001
Training Cost 0.48897195825975626
Training accuracy 91.23125


Training Cost 0.47603200920798483
Training accuracy 91.5
Training Cost 0.47599292339749893
Training accuracy 91.5
Training Cost 0.4759549599406115
Training accuracy 91.48125
Training Cost 0.47591812027783725
Training accuracy 91.46875
Training Cost 0.47588242814455534
Training accuracy 91.475
Training Cost 0.4758479302419697
Training accuracy 91.46875
Training Cost 0.47581469631652806
Training accuracy 91.46875
Training Cost 0.47578281883120405
Training accuracy 91.5
Training Cost 0.4757524122613556
Training accuracy 91.5125
Training Cost 0.4757236118474248
Training accuracy 91.5125
Training Cost 0.4756965714473713
Training accuracy 91.5
Training Cost 0.47567146001422866
Training accuracy 91.50625
Training Cost 0.47564845622600854
Training accuracy 91.475
Training Cost 0.4756277409421646
Training accuracy 91.46875
Training Cost 0.475609487452655
Training accuracy 91.46875
Training Cost 0.47559384989207815
Training accuracy 91.4625
Training Cost 0.4755809506477004
Training accuracy 91.4

Training accuracy 91.44375
Training Cost 0.4755522901116344
Training accuracy 91.43124999999999
Training Cost 0.4755593475161666
Training accuracy 91.425
Training Cost 0.4755658754858369
Training accuracy 91.43124999999999
Training Cost 0.47557189004866723
Training accuracy 91.44375
Training Cost 0.4755774089662573
Training accuracy 91.4375
Training Cost 0.4755824513029614
Training accuracy 91.4375
Training Cost 0.47558703703596744
Training accuracy 91.4375
Training Cost 0.47559118671336215
Training accuracy 91.425
Training Cost 0.475594921164121
Training accuracy 91.40625
Training Cost 0.47559826126068194
Training accuracy 91.3875
Training Cost 0.47560122773144464
Training accuracy 91.3875
Training Cost 0.47560384101738107
Training accuracy 91.39375
Training Cost 0.4756061211642331
Training accuracy 91.3875
Training Cost 0.47560808773981184
Training accuracy 91.39375
Training Cost 0.47560975976501113
Training accuracy 91.4125
Training Cost 0.4756111556474722
Training accuracy 91.41875